# Running vLLM

This provides the model endpoint needed for Q&A generation in the InstructLab pipeline.

## Install vLLM

In [1]:
# Install vLLM
!pip install vllm huggingface-hub python-dotenv

## Download Model


In [2]:
import os

# hf_repo = "RedHatAI/Llama-3.3-70B-Instruct-FP8-dynamic"
# hf_repo = "RedHatAI/gpt-oss-120b"
# hf_repo = "RedHatAI/phi-4-FP8-dynamic"
hf_repo = "RedHatAI/phi-4"
# hf_repo = "RedHatAI/gpt-oss-120b"


model_dir = f"models/{hf_repo}"


In [3]:
!mkdir -p $model_dir
!huggingface-cli download $hf_repo --local-dir $model_dir

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 20 files: 100%|██████████████████████| 20/20 [00:00<00:00, 2978.27it/s]
/opt/app-root/src/red-hat-ai-examples/examples/knowledge-tuning/models/RedHatAI/phi-4


## Run vLLM Server - Foreground

Run this cell to start the server in foreground mode. The server will run until you interrupt the kernel.

In [4]:
served_model_name = "teacher"
num_gpus="2"
max_model_len="16000"

In [ ]:
!vllm serve $model_dir \
    --host 0.0.0.0 \
    --port 8000 \
    --served-model-name $served_model_name \
    --tensor-parallel-size=$num_gpus \
    --distributed-executor-backend=mp \
    --max-model-len=$max_model_len

INFO 11-11 17:51:17 [__init__.py:216] Automatically detected platform cuda.
(APIServer pid=3286) INFO 11-11 17:51:21 [api_server.py:1839] vLLM API server version 0.11.0
(APIServer pid=3286) INFO 11-11 17:51:21 [utils.py:233] non-default args: {'model_tag': 'models/RedHatAI/phi-4', 'host': '0.0.0.0', 'model': 'models/RedHatAI/phi-4', 'max_model_len': 16000, 'served_model_name': ['teacher'], 'distributed_executor_backend': 'mp', 'tensor_parallel_size': 2}
(APIServer pid=3286) INFO 11-11 17:51:21 [model.py:547] Resolved architecture: Phi3ForCausalLM
(APIServer pid=3286) `torch_dtype` is deprecated! Use `dtype` instead!
(APIServer pid=3286) INFO 11-11 17:51:21 [model.py:1510] Using max model len 16000
(APIServer pid=3286) INFO 11-11 17:51:23 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 11-11 17:51:26 [__init__.py:216] Automatically detected platform cuda.
(EngineCore_DP0 pid=3424) INFO 11-11 17:51:30 [core.py:644] Waiting for init message from front-

Test with a small model

In [6]:
# !vllm serve Qwen/Qwen2.5-0.5B-Instruct \
#     --host 0.0.0.0 \
#     --port 8000 \
#     --max-model-len 2048